In [ ]:
# requests will allow us to make a web request
import requests
# BeautifulSoup will allow us to easily parse the website's HTML code
from bs4 import BeautifulSoup

In [ ]:
# Setup a list of lists to temporarily store the data
my_list = [['Ep_Title','Rating','Season','Episode']]
# Set a "base" URL that we can append onto
base_url = "https://www.imdb.com/title/tt0096697/episodes"
# Set the first URL that we hope to access
next_url = "https://www.imdb.com/title/tt0096697/episodes?season=1"
# Loop through three seasons of The Simpsons
for season in range(1,4):
    # Make an external web request and pull the website's code
    source = requests.get(next_url).text
    # Parse the website's code into a nested data structure
    soup = BeautifulSoup(source, 'lxml')
    temp = soup.find('h3',id='episode_top')
    season_str=str(temp.string)
    season_int=int(season_str[-1])
    # After investigating the HTML code, I figured out where to find
    # the rating, title, and episode for each episode.  So, I grabbed that data
    # and then appended it to "my_list"
    for episode in soup.find_all('div',class_="list_item odd"):
        rating = episode.find('span',class_="ipl-rating-star__rating")
        temp = episode.find('a')
        title = temp.get('title')
        ep = episode.meta['content']
        my_list.append([title,float(rating.text),season_int,int(ep)])
    # Do the same thing as above but for the "even" episodes on this page
    for episode in soup.find_all('div',class_="list_item even"):
        rating = episode.find('span',class_="ipl-rating-star__rating")
        temp = episode.find('a')
        title = temp.get('title')
        ep = episode.meta['content']
        my_list.append([title,float(rating.text),season_int,int(ep)])
    # Update the web address for the next webpage to be accessed
    # (There is a better way to do this using "urllib.parse")
    next_url = base_url + soup.find(id='load_next_episodes').get('href')

In [ ]:
import numpy as np
import pandas as pd
# Get the data into a pandas DataFrame
col_headers = my_list.pop(0)
df = pd.DataFrame(my_list, columns=col_headers)
df

In [ ]:
import pymysql
pymysql.install_as_MySQLdb()
%reload_ext sql
%sql mysql://student:@localhost/
%sql USE simpsons;

In [ ]:
%%sql
SELECT     *
FROM       episodes
WHERE      season <= 3
LIMIT      5;

In [ ]:
query_result = %sql SELECT * FROM episodes WHERE season <= 3;
simpsons_df = query_result.DataFrame()
simpsons_merged = pd.merge(simpsons_df, df, left_on=['season','number_in_season'], right_on=['Season','Episode'], how='inner')
simpsons_merged[['title','original_air_date','season','number_in_season','Rating']]

In [ ]:
simpsons_merged.groupby(['season']).size()